In [20]:
%pip install pip install -q -U google-genai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for install


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
from typing import Optional, Literal
from pydantic import BaseModel, Field
from google import genai
import os
from dotenv import load_dotenv
import logging

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# Cargamos nuestra API_KEY de Gemini
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Se define la variable global del modelo a utilizar en las requests
model="gemini-2.0-flash"

client = genai.Client(api_key=API_KEY)
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works"
)

2025-03-05 13:14:53 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 13:15:15 - INFO - AFC remote call 1 is done.


In [22]:

# --------------------------------------------------------------
# Step 1: Definimos las estruturas de datos para enrutar al modelo
# --------------------------------------------------------------
conf_score_descrp = "Puntuación de confianza del 0 al 1"

class PrimaryRequestType(BaseModel):
    """ Llamada enrutadora a la LLM: Contiene qué tipo de solitud primaria está siendo realizada"""

    request_type: Literal["calendario", "extrac_inf","otro"] = Field(
        description="Tipo de solicitud primaria realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Descripción de la solicitud")

class CalendarRequestType(BaseModel):
    """Llamada enrutadora a la LLM: Contiene qué tipo de solicitud de calendario está siendo realizada"""

    request_type: Literal["nuevo_evento", "modif_evento", "otro"] = Field(
        description="Tipo de solicitud de calendario realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Cleaned description of the request")


class NewEventDetails(BaseModel):
    """Detalles para la creación de un nuevo evento"""

    name: str = Field(description="Nombre del evento")
    date: str = Field(description="Fecha y hora del evento. (ISO 8601)")
    duration_minutes: int = Field(description="Duración en minutos")
    participants: list[str] = Field(description="Lista de participantes")


class Change(BaseModel):
    """Detalles a cambiar del evento existente"""

    field: str = Field(description="Campo a cambiar")
    new_value: str = Field(description="Nuevo valor a cambiar")


class ModifyEventDetails(BaseModel):
    """Detalles a modificar un evento existente"""

    event_identifier: str = Field(
        description="Descripción para identificar el evento existente"
    )
    changes: list[Change] = Field(description="Lista de cambios a realizar")
    participants_to_add: list[str] = Field(description="Nuevo participante a añadir")
    participants_to_remove: list[str] = Field(description="Participantes a remover")


class CalendarResponse(BaseModel):
    """Formato de respuesta final"""

    success: bool = Field(description="Si la operación tuvo éxito")
    message: str = Field(description="Mensaje de respuesta amistable al usuario")
    calendar_link: Optional[str] = Field(description="Link de calendario si aplica")


# --------------------------------------------------------------
# Step 2: Define el enrutamiento y las funciones de procesamiento 
# --------------------------------------------------------------

def route_primary_request(user_input: str) -> PrimaryRequestType:
    """ Enruta la llamada a la LLM para determinar el tipo de solicitud"""
    logger.info("Enrutando la solicitud primaria")

    client = genai.Client(api_key=API_KEY)
    response = client.models.generate_content(
        model=model,
        contents=[
            {
                "rol": "sistema",
                "contenido":"Determina si la solicitud del usuario está relacionada con los calendarios o con extracción de información"
            },
            {
                "rol" : "usuario",
                "contenido" : user_input
            }
        ],
        config={
            'response_schema': PrimaryRequestType,
        },
    )
    logger.info(f"Solicitud enrutada como: {response.request_type} con una confianza de: {response.confidence}")
    return response
    

def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Enruta la llamada a la LLM para determinar el tipo de solicitud de calendario"""
    logger.info("Enrutando solicitud de calendario")

    client = genai.Client(api_key=API_KEY)
    response = client.models.generate_content(
        model=model,
        contents=[
            {
                "rol": "sistema",
                "contenido":"Determina si esta es una solicitud para crear un nuevo evento de calendario o modificar un evento existente"
            },
            {
                "rol" : "usuario",
                "contenido" : user_input
            }
        ],
        config={
            'response_schema': CalendarRequestType,
        },
    )
    
    logger.info(
        f"Solicitud enrutada como: {response.request_type} con una confianza de: {response.confidence}"
        )
    return response


def handle_new_event(description: str) -> CalendarResponse:
    """Procesa un nuevo evento"""
    logger.info("Procesando una solicitud de un nuevo evento")

    # Obtener los detalles del evento
    client = genai.Client(api_key=API_KEY)
    response = client.models.generate_content(
        model=model,
        contents=[
            {
                "rol": "sistema",
                "contenido":"Extrae los datos para la creación de un nuevo evento."
            },
            {
                "rol" : "usuario",
                "contenido" : description
            }
        ],
        config={
            'response_schema': NewEventDetails,
        },
    )
    
    logger.info(
        f"Nuevo evento: {response.model_dump_json(indent=2)}"
        )

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Creado un nuevo evento '{response.name}' para {response.date} con {', '.join(response.participants)}",
        calendar_link=f"calendar://new?event={response.name}",
    )


def handle_modify_event(description: str) -> CalendarResponse:
    """Procesa la modificación de un evento"""
    logger.info("Procesando una solicitud de modificación de evento")

    # Obtener detalles para modificación

    client = genai.Client(api_key=API_KEY)
    response = client.models.generate_content(
        model=model,
        contents=[
            {
                "rol": "sistema",
                "contenido":"Extrae los datos para la modificación de un evento existente."
            },
            {
                "rol" : "usuario",
                "contenido" : description
            }
        ],
        config={
            'response_schema': ModifyEventDetails,
        },
    )

    logger.info(
        f"Nuevo evento: {response.model_dump_json(indent=2)}"
        )

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Modificando evento '{response.name}' para {response.date} con {', '.join(response.participants)}",
        calendar_link=f"calendar://new?event={response.name}",
    )



def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Procesando solicitud de calendario")

    # Enrutar la solicitud
    route_result = route_calendar_request(user_input)

    # Eluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None

    # Enrutar el handler apropiado
    if route_result.request_type == "nuevo_evento":
        return handle_new_event(route_result.description)
    elif route_result.request_type == "modif_evento":
        return handle_modify_event(route_result.description)
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None


def process_primary_request(user_input: str):
    """Funcion principal para implementar el enrutamiento del trabajo"""
    logger.info("Procesando solicitud primaria")

    # Enrutar la solicitud
    route_result = route_primary_request(user_input)

    # Evaluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None
    
    # Enrutar el handler apropiado
    if route_result.request_type == "calendario":
        return process_calendar_request(route_result.description)
    elif route_result.request_type == "extrac_inf":
        logger.warning("Handler para funcion no implementado")
        return None
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None

# --------------------------------------------------------------
# Step 3: Probar con nuevo evento
# --------------------------------------------------------------

new_event_input = "Programemos una reunión de equipo el próximo martes a las 2 p. m. con Alice y Bob."
result = process_calendar_request(new_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 4: Probar con modificar un evento
# --------------------------------------------------------------

modify_event_input = (
    "¿Podrías trasladar la reunión del equipo con Alice y Bob al miércoles a las 3 p. m.?"
)
result = process_calendar_request(modify_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 5: Probar con input inválido
# --------------------------------------------------------------

invalid_input = "¿Cómo está el tiempo hoy?"
result = process_calendar_request(invalid_input)
if not result:
    print("Solicitud no reconocida como operación de calendario")

2025-03-05 13:15:16 - INFO - Procesando solicitud de calendario
2025-03-05 13:15:16 - INFO - Enrutando solicitud de calendario
2025-03-05 13:15:16 - INFO - AFC is enabled with max remote calls: 10.


ValidationError: 35 validation errors for _GenerateContentParameters
contents.list[union[Content,list[union[File,Part,is-instance[Image],str]],File,Part,is-instance[Image],str]].0.Content.rol
  Extra inputs are not permitted [type=extra_forbidden, input_value='sistema', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
contents.list[union[Content,list[union[File,Part,is-instance[Image],str]],File,Part,is-instance[Image],str]].0.Content.contenido
  Extra inputs are not permitted [type=extra_forbidden, input_value='Determina si esta es una...car un evento existente', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
contents.list[union[Content,list[union[File,Part,is-instance[Image],str]],File,Part,is-instance[Image],str]].0.list[union[File,Part,is-instance[Image],str]]
  Input should be a valid list [type=list_type, input_value={'rol': 'sistema', 'conte...ar un evento existente'}, input